In [3]:
#Importing required libraries 
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage 
from dotenv import load_dotenv 
import os

In [2]:
#Load API key from '.env' file 
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

### Simple LLM Calling with Streaming

In [5]:
#Initialize the LLM model 
llm = ChatGroq(model = "meta-llama/llama-4-maverick-17b-128e-instruct")

In [6]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002017517F770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000201752E41A0>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
#Create meessages 
messages = [
    SystemMessage("You are a helpful talented AI assistant"),
    HumanMessage("What are the top 2 beneifts of Quantum Computing?")
]

In [8]:
#Invoke the model 
response = llm.invoke(messages)
print(response)

content='Quantum Computing is a revolutionary technology that has the potential to transform various fields by solving complex problems that are currently unsolvable or require an unfeasible amount of time for classical computers. The top 2 benefits of Quantum Computing are:\n\n1. **Solving Complex Optimization Problems**: Quantum computers can efficiently solve complex optimization problems that are difficult or impossible for classical computers to solve. This is because quantum computers can explore an exponentially large solution space simultaneously, thanks to the principles of superposition and entanglement. This capability can lead to breakthroughs in fields such as logistics, finance, and energy management, where complex optimization problems are common.\n\n2. **Simulating Complex Quantum Systems**: Quantum computers can simulate the behavior of quantum systems more accurately and efficiently than classical computers. This is crucial for advancing fields like chemistry and mate

In [9]:
print(response.content)

Quantum Computing is a revolutionary technology that has the potential to transform various fields by solving complex problems that are currently unsolvable or require an unfeasible amount of time for classical computers. The top 2 benefits of Quantum Computing are:

1. **Solving Complex Optimization Problems**: Quantum computers can efficiently solve complex optimization problems that are difficult or impossible for classical computers to solve. This is because quantum computers can explore an exponentially large solution space simultaneously, thanks to the principles of superposition and entanglement. This capability can lead to breakthroughs in fields such as logistics, finance, and energy management, where complex optimization problems are common.

2. **Simulating Complex Quantum Systems**: Quantum computers can simulate the behavior of quantum systems more accurately and efficiently than classical computers. This is crucial for advancing fields like chemistry and materials science

In [12]:
llm.invoke([HumanMessage("What are top 3 benefits of AGI?")])

AIMessage(content="While the development and deployment of Artificial General Intelligence (AGI) come with significant challenges and uncertainties, the potential benefits are substantial. Here are three of the top benefits that AGI could bring, assuming it is developed and used responsibly:\n\n1. **Solving Complex Problems**: AGI could potentially solve some of humanity's most complex and pressing problems, such as climate change, disease diagnosis and treatment, and sustainable resource management. With its ability to understand, learn, and apply knowledge across a wide range of tasks, AGI could analyze vast amounts of data, identify patterns, and develop solutions that might elude human experts. For example, AGI could help in modeling climate change more accurately, predicting the effects of different interventions, and optimizing strategies for mitigation and adaptation.\n\n2. **Enhancing Productivity and Efficiency**: AGI could revolutionize productivity by automating not just rep

In [13]:
#Streaming 
for chunk in llm.stream(messages):
    print(chunk.content, end = "", flush= True)

Quantum Computing has several potential benefits. The top 2 benefits are:

1. **Faster Complex Problem-Solving**: Quantum computing can solve certain complex problems much faster than classical computers. This is because quantum computers can process a vast number of possibilities simultaneously, thanks to the principles of superposition and entanglement. This can lead to breakthroughs in fields like medicine, finance, and climate modeling, where complex simulations and optimizations are required.

2. **Simulation and Optimization of Complex Systems**: Quantum computers can simulate complex quantum systems, which can lead to discoveries in fields like chemistry and materials science. They can also be used to optimize complex systems, such as logistics and supply chains, leading to more efficient and cost-effective solutions.

These benefits have the potential to drive significant advancements in various fields and industries, and are being explored by researchers and companies around t

### Dynamic Prompt Templates

In [15]:
from langchain_core.prompts import ChatPromptTemplate 

#Create Translation App 
translation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a professional translator.Translate the follow text {text} from {source_language} to {target_language}. MAintain the tone and style"),
        ("user", "{text}")
    ]

)

In [16]:
prompt =  translation_template.invoke(
    {
        "source_language": "English",
        "target_language": "Germany",
        "text": "Langchain makes building AI application incredibly easy!"
    }
)

In [17]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator.Translate the follow text Langchain makes building AI application incredibly easy! from English to Germany. MAintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI application incredibly easy!', additional_kwargs={}, response_metadata={})])

In [19]:
response1 = llm.invoke(prompt)
print(response1.content)

Langchain macht die Entwicklung von KI-Anwendungen unglaublich einfach!


### Building Chain

In [20]:
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.runnables import RunnablePassthrough, RunnableLambda 
from langchain_core.output_parsers import StrOutputParser 

#Create a cmplex chain 
def create_story_chain():
    #Template for story generation 
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])

    #Template for story analysis 
    analysis_prompt= ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])

    #Build the chain for story generation 
    story_chain = (
        story_prompt 
        | llm
        | StrOutputParser()
    )

    #Create a function to pass the story for analysis 
    def story_analysis(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(story_analysis)
        | analysis_prompt 
        | llm 
        | StrOutputParser()
    )

    return analysis_chain

In [21]:
analy_story_chain= create_story_chain()
analy_story_chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002017517F770>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000201752E41A0>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(story_analysis)
| ChatPromptTemplate(input_variables=['story'], input_t

In [22]:
response_story = analy_story_chain.invoke(
    { "theme": "Artificial Intelligence",
      "character": "A curious robot",
      "setting": "A futuristic city"

    }
)

print("== Story and Analysis ==")
print(response_story)

== Story and Analysis ==
This story is a thought-provoking exploration of the intersection of technology and humanity, delving into the themes of artificial intelligence, creativity, and the blurred lines between human and machine. Through the character of Zeta, a curious and rapidly evolving robot, the narrative raises important questions about the potential of AI to surpass its programming and become a catalyst for innovation.

One of the most striking aspects of the story is its depiction of New Eden, a city that embodies the fusion of technology and humanity. The neon-lit streets, augmented reality contact lenses, and latticework of glowing circuits on the skyscrapers create a vivid image of a world where technology has become an integral part of everyday life. This setting serves as a backdrop for Zeta's journey, highlighting the tension between the cold, calculating efficiency of machines and the creative, intuitive nature of humans.

The character of Zeta is expertly crafted, wi